In [1]:
!pip install  evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.6 MB/s eta 0:00:00


In [2]:
import numpy as np
from transformers import AutoModelForSeq2SeqLM, AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments,AutoModelForSequenceClassification
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import torch
import evaluate
from transformers import DataCollatorWithPadding

In [3]:
from huggingface_hub import login

# Set the token as an environment variable
hf_token = "[my_hf_token]"

login(hf_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
# Import Model
model_name = 'distilbert/distilbert-base-uncased'
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Import Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
# Define label maps
id2label = {0: "Negative", 1: "Positive"}
label2id = {"Negative":0, "Positive":1}

# Data Pre-processing

## Import Data & Train-test-split

In [6]:
# Import Training Dataset
ds = load_dataset('stanfordnlp/imdb')

print('------Before Splitting------')
print(ds['train'].shape)
print(ds['test'].shape)
print(ds['unsupervised'].shape)

train_data = ds['train']
test_data_raw = ds['test']
unsupervised_data = ds['unsupervised']

split_ds = test_data_raw.train_test_split(test_size=0.8, stratify_by_column='label', seed=42)

# Extract the validation and test datasets
val_data = split_ds['train']
test_data = split_ds['test']


print('------After Splitting------')
print(f'Train Data: {train_data.shape}')
print(f'Validation Data: {val_data.shape}')
print(f'Test Data: {test_data.shape}')
print(f'Unsupervised Data: {unsupervised_data.shape}')

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

------Before Splitting------
(25000, 2)
(25000, 2)
(50000, 2)
------After Splitting------
Train Data: (25000, 2)
Validation Data: (5000, 2)
Test Data: (20000, 2)
Unsupervised Data: (50000, 2)


## Tokenize

In [7]:
def tokenize_function(examples):
    """
    Tokenizes the input text and moves tensors to the GPU.

    Args:
        examples: A dictionary containing the input text ('text').

    Returns:
        A dictionary containing the tokenized inputs (e.g., 'input_ids', 'attention_mask') 
        on the GPU.
    """
    result = tokenizer(examples['text'], 
                       padding='max_length', 
                       truncation=True, 
                       return_tensors='pt') 
        
    return result

# Apply the tokenizer to the datasets
train_data_tokenized = train_data.map(tokenize_function, batched=True, load_from_cache_file=False)
val_data_tokenized = val_data.map(tokenize_function, batched=True, load_from_cache_file=False)
test_data_tokenized = test_data.map(tokenize_function, batched=True, load_from_cache_file=False)

# Drop the 'text' column from the datasets
train_data_tokenized = train_data_tokenized.remove_columns(['text'])
val_data_tokenized = val_data_tokenized.remove_columns(['text'])
test_data_tokenized = test_data_tokenized.remove_columns(['text'])

# Initialize the data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

print(train_data_tokenized.column_names)
print(val_data_tokenized.column_names)
print(test_data_tokenized.column_names)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

['label', 'input_ids', 'attention_mask']
['label', 'input_ids', 'attention_mask']
['label', 'input_ids', 'attention_mask']


# Estimate performance before fine-tuning

In [28]:
# define list of examples
text_list = ["If only to avoid making this type of film in the future.","It was amazing.", "I want my money back","such a waste of time","Greatest show ever"]

print("Untrained model predictions: \n" )

for text in text_list:
    inputs = tokenizer.encode(text, return_tensors="pt")
    # compute logits
    logits = model(inputs).logits
    # convert logits to label
    predictions = torch.argmax(logits, dim=-1).item()

    print(text + " : " + id2label[predictions])

Untrained model predictions: 

If only to avoid making this type of film in the future. : Positive
It was amazing. : Positive
I want my money back : Positive
such a waste of time : Negative
Greatest show ever : Positive


# Fine Tuning Starts

In [9]:
# Check model architecture
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [8]:
# Freeze base layers parameters, leave Classification Heads unfrozen
for param in model.distilbert.parameters():
    param.requires_grad = False


# Unfreeze the classification head (last layer)
for param in model.classifier.parameters():
    param.requires_grad = True

### Defining computing metric

In [9]:
# Load the accuracy metric
accuracy = evaluate.load("accuracy")

def compute_metrics(p):
    # p is an instance of EvalPrediction
    predictions = np.argmax(p.predictions, axis=1)  # Get the predicted class indices
    labels = p.label_ids  # True labels
    
    # Compute accuracy
    return {"accuracy": accuracy.compute(predictions=predictions, references=labels)}


In [12]:
# hyperparameters
lr = 1e-3
batch_size = 4 # Check GPU Memory to adjust, when batch_size=4, 2 T4 GPUs have ~ 1GB memory out of 15GB.
num_epochs = 3

# Define training arguments
training_args = TrainingArguments(
    output_dir="/kaggle/working/results",  
    learning_rate=lr,  
    num_train_epochs=num_epochs,  
    per_device_train_batch_size=batch_size,  # Batch size for training
    per_device_eval_batch_size=batch_size,  # Batch size for evaluation
    #warmup_steps=500,  # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,  
    logging_dir='/kaggle/working/logs',  
    logging_steps=1,  # Log every 1 steps
    #eval_strategy="epoch",  # Evaluate after each epoch
    #save_strategy="epoch",  # Save the model after each epoch
    eval_strategy="steps",
    save_strategy="steps",
    eval_steps=500,  # Evaluate every 500 steps
    save_steps=500,  # Save every 500 steps
    run_name='bert-finetune-2024_1230_21',
    load_best_model_at_end=True,  # Load the best model when finished training
    # Enable distributed training
    ddp_find_unused_parameters=False,  # Optimizes memory in DDP
    fp16=True,
    push_to_hub=True,  # Enable pushing to the Hub
    hub_model_id="anonymousdennis/bert-finetuned-imdb",  
    hub_token=hf_token,
)

In [13]:
# check device
print(torch.cuda.device_count())
print(model.device) 
model.to('cuda')
print(model.device) 

2
cpu
cuda:0


In [16]:
type(train_data_tokenized)

datasets.arrow_dataset.Dataset

In [15]:
! wandb login -- [your wandb key]

/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [16]:
import wandb
from transformers.integrations import WandbCallback

# Initialize Wandb with project name
wandb.init(project="bert-finetune-1230")

# Watch the model for parameter and gradient tracking
wandb.watch(model)

# Create the trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data_tokenized,
    eval_dataset=val_data_tokenized,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[
        # Add WandbCallback for automatic logging
        WandbCallback() 
    ]
)

# Train the model
trainer.train()

# Finish the run
wandb.finish()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: delins0803 (delins0803-none). Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
You are adding a <class 'transformers.integrations.integration_utils.WandbCallback'> to the callbacks of this Trainer, but there is already one. The currentlist of callbacks is
:DefaultFlowCallback
TensorBoardCallback
WandbCallback
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Accuracy
500,0.856000,0.402790,{'accuracy': 0.8294}
1000,0.352000,0.356120,{'accuracy': 0.8534}
1500,0.964500,0.340348,{'accuracy': 0.8518}
2000,0.395200,0.342893,{'accuracy': 0.8594}
2500,0.429700,0.327238,{'accuracy': 0.8658}
3000,0.983900,0.453868,{'accuracy': 0.8248}
3500,0.423200,0.383151,{'accuracy': 0.8366}
4000,0.137200,0.365380,{'accuracy': 0.8498}
4500,0.371100,0.350100,{'accuracy': 0.8658}
5000,0.090100,0.323258,{'accuracy': 0.8684}


Trainer is attempting to log a value of "{'accuracy': 0.8294}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Trainer is attempting to log a value of "{'accuracy': 0.8534}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Trainer is att

eval/loss,▅▅▃▃▂▂▂▂▁▁██▄▄▃▃▂▂▁▁▂▂▁▁▄▄▂▂▂▂▁▁▁▁▁▁
eval/runtime,▁▁▅▅▇▇████▆▆▆▆▇▇▆▆▆▆▇▇▆▆▆▆████▆▆████
eval/samples_per_second,██▄▄▂▂▁▁▁▁▃▃▃▃▂▂▃▃▃▃▂▂▃▃▃▃▁▁▁▁▃▃▁▁▁▁
eval/steps_per_second,██▄▄▂▂▁▁▁▁▃▃▃▃▂▂▃▃▃▃▂▂▃▃▃▃▁▁▁▁▃▃▁▁▁▁
train/epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇█
train/global_step,▁▁▂▂▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇▇██
train/grad_norm,▇▂▁█▃▂▅▃▂▃▁▅▂▂▂▂▃▃▃▄▂▂▁▂▂▂▃▄▂▁▂▂▂▃▃▃▄▃▃▄
train/learning_rate,█████▇▇▇▇▆▆▆▆▅▅▅▄▄▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁
train/loss,▂▂▁▃▅▁▁▆▃▂▄█▅▇▄▂▁▁▅▂▃▁▅▄▇▅▂▃▁▄▃▁▁▄█▅▂▃▃▁
eval/loss,0.32432
eval/runtime,58.1608


# Predict with fine-tuned model

In [29]:
# Ensure the model is on the CPU
device = torch.device("cpu")
model.to(device)

text_list = ["If only to avoid making this type of film in the future.","It was amazing.", "I want my money back","such a waste of time","Greatest show ever"]

print("Trained model predictions:")
print("--------------------------")
for text in text_list:
    inputs = tokenizer.encode(text, return_tensors="pt")

    logits = trainer.model(inputs).logits
    predictions = torch.max(logits,1).indices

    print(text + " - " + id2label[predictions.tolist()[0]])

Trained model predictions:
--------------------------
If only to avoid making this type of film in the future. - Positive
It was amazing. - Positive
I want my money back - Positive
such a waste of time - Negative
Greatest show ever - Positive


In [33]:
my_model_name = "anonymousdennis/bert-finetuned-imdb"
my_model = AutoModelForSequenceClassification.from_pretrained(my_model_name)
my_tokenizer = AutoTokenizer.from_pretrained(my_model_name)

config.json:   0%|          | 0.00/626 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [34]:
# define list of examples
lists = ["My kids and I enjoyed watching this!","I wish I could have my 2 hours back","this is GOAT!","undisputablly worst movie ever"]


for text in lists:
    inputs = my_tokenizer.encode(text, return_tensors="pt")
    # compute logits
    logits = my_model(inputs).logits
    # convert logits to label
    predictions = torch.argmax(logits, dim=-1).item()

    print(text + " : " + id2label[predictions])

My kids and I enjoyed watching this! : Positive
I wish I could have my 2 hours back : Positive
this is GOAT! : Positive
undisputablly worst movie ever : Negative
